<a href="https://colab.research.google.com/github/FaiqNasir525/Custom_CNN_Based_Image_Classification_Model/blob/main/CNN_Image_Classification_with_Advanced_Hyperparameter_Tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Image Classification Using CNN with the CIFAR-10 Dataset
using **basic Hyperparameter Tunning**

In [ ]:
!pip install keras_tuner

# Step 1: Importing necessary libraries
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers
from keras_tuner.tuners import RandomSearch
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
# Step 2: Loading and preprocessing the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

# Step 3: Defining the class names for CIFAR-10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
# Step 4: Define a function to build the model.
def build_model(hp):
    model = models.Sequential()

    # Tune the number of convolutional layers (1, 2 or 3)
    for i in range(hp.Int('conv_layers', 1, 3)):
        if i == 0:
            model.add(layers.Conv2D(
                filters=hp.Int('filters_' + str(i), min_value=32, max_value=128, step=16),
                kernel_size=3,
                activation='relu',
                kernel_regularizer=regularizers.l2(1e-4),
                padding='same',
                input_shape=(32, 32, 3)))
        else:
            for j in range(hp.Int('deep_conv_layers', 1, 3)):
                model.add(layers.Conv2D(
                    filters=hp.Int('filters_' + str(i) + str(j), min_value=64, max_value=128, step=16),
                    kernel_size=3,
                    activation='relu',
                    kernel_regularizer=regularizers.l2(1e-4),
                    padding='same'))
                model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Dropout(rate=hp.Float(f'dropoutConv_{i}', 0.0, 0.5, step=0.1)))

    model.add(layers.Flatten())

    # Tune the number of dense layers (1, 2, or 3)
    for i in range(hp.Int('dense_layers', 1, 3)):
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=32, max_value=128, step=16),
            activation='relu'))

        # Tune the dropout rate
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(rate=hp.Float(f'dropoutDense_{i}', 0.0, 0.5, step=0.1)))

    # The last dense layer with 10 output units (for 10 classes)
    model.add(layers.Dense(10, activation='softmax'))

    # Choose an optimizer and learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))

    model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

    return model

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
1                 |1                 |conv_layers
80                |80                |filters_0
0                 |0                 |dropoutConv_0
1                 |1                 |dense_layers
48                |48                |units_0
0.4               |0.4               |dropoutDense_0
0.0001            |0.0001            |learning_rate

Epoch 1/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 86s 107ms/step - accuracy: 0.3019 - loss: 1.9703 - val_accuracy: 0.4478 - val_loss: 1.5662
Epoch 2/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 134s 97ms/step - accuracy: 0.4833 - loss: 1.5055 - val_accuracy: 0.5668 - val_loss: 1.3168
Epoch 3/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 83s 99ms/step - accuracy: 0.5379 - loss: 1.3547 - val_accuracy: 0.5776 - val_loss: 1.2349
Epoch 4/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 98ms/step - accuracy: 0.5665 - loss: 1.2630 - val_accuracy: 0.5752 - val_loss: 1.2417
Epoch 5/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 9

KeyboardInterrupt: 

In [ ]:
# Step 5: Define the Tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=1,
    directory='my_dir',
    project_name='cifar10_tunning'
)

# Step 6: Perform the Hyperparameter search
tuner.search(train_images, train_labels, batch_size=64, epochs=15, validation_data=(test_images, test_labels))

# Step 7: Get the best Hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
# Step 8: Print the all the best hyperparameters
print(f"Best number of convolutional layers: {best_hps.get('conv_layers')}")
print(f"Best number of deep convolutional layers: {best_hps.get('deep_conv_layers')}")
print(f"Best number of dense layers: {best_hps.get('dense_layers')}")
print(f"Best learning rate: {best_hps.get('learning_rate')}")
for i in range(best_hps.get('conv_layers')):
  print(f"Best number of filters for the convolutional layer {i+1}: {best_hps.get(f'filters_{i}')}")
  print(f"Best dropout rate for the convolutional layer {i+1}: {best_hps.get(f'dropoutConv_{i}')}")
  if best_hps.get('deep_conv_layers') > 1:
    for j in range(best_hps.get('deep_conv_layers')):
      print(f"Best number of filters for the deep convolutional layer {i+1}.{j+1}: {best_hps.get(f'filters_{i}{j}')}")

for i in range(best_hps.get('dense_layers')):
  print(f"Best number of units for the dense layer {i+1}: {best_hps.get(f'units_{i}')}")
  print(f"Best dropout rate for the dense layer {i+1}: {best_hps.get(f'dropout_{i}')}")

Epoch 1/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.3123 - loss: 1.9830 - val_accuracy: 0.5523 - val_loss: 1.2443
Epoch 2/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 25s 6ms/step - accuracy: 0.5399 - loss: 1.2846 - val_accuracy: 0.6446 - val_loss: 0.9967
Epoch 3/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.6229 - loss: 1.0733 - val_accuracy: 0.6050 - val_loss: 1.1257
Epoch 4/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.6694 - loss: 0.9435 - val_accuracy: 0.6495 - val_loss: 1.0105
Epoch 5/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.6977 - loss: 0.8714 - val_accuracy: 0.5892 - val_loss: 1.2380
Epoch 6/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.7211 - loss: 0.8130 - val_accuracy: 0.6687 - val_loss: 0.9497
Epoch 7/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.7327 - loss: 0.7683 - val_accuracy: 0.7345 - val_loss: 0.7527
Epoch 8/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7518 - loss: 0.7249 - va

KeyboardInterrupt: 

In [ ]:
# Step 9: Build the model with the best Hyperparameters and train it
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_images, train_labels, batch_size=64, epochs=100, validation_data=(test_images, test_labels))

In [ ]:

# Step 10: Plotting training & validation accuracy and loss values
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.grid()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.grid()

plt.show()

In [ ]:
#testing
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)

313/313 - 1s - 3ms/step - accuracy: 0.8172 - loss: 0.6956
0.8172000050544739
